In [ ]:
import pandas as pd

# Baca data dari file CSV
df_tinggi_air = pd.read_csv('tinggi_air.csv')
df_tinggi_air['tanggal'] = pd.to_datetime(df_tinggi_air['tanggal'])

df_rainfall = pd.read_csv('laporan_iklim_harian.csv')
df_rainfall['Tanggal'] = pd.to_datetime(df_rainfall['Tanggal'])

# Cek duplikasi pada kolom indeks sebelum penggabungan
print(df_tinggi_air['tanggal'].duplicated().any())
print(df_rainfall['Tanggal'].duplicated().any())

True
False


<ipython-input-2-4c8e173e5877>:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_rainfall['Tanggal'] = pd.to_datetime(df_rainfall['Tanggal'])


In [ ]:
# Hapus duplikasi pada kolom indeks df_tinggi_air
df_tinggi_air = df_tinggi_air[~df_tinggi_air['tanggal'].duplicated()]

# Merge data menggunakan pd.concat
df_combined = pd.concat([df_tinggi_air.set_index('tanggal'), df_rainfall.set_index('Tanggal')], axis=1, join='outer')
df_combined = df_combined.fillna(0)
df_combined = df_combined.interpolate(method='linear')

# Tampilkan DataFrame hasil penggabungan setelah interpolasi
print(df_combined)

                      nama_pintu_air    lokasi    latitude    longitude  \
2019-12-31 23:40:00  P.A. Istiqlal    Ciliwung  -6,168,254  106,831,350   
2020-01-01 00:00:00                0         0           0            0   
2020-01-01 00:10:00  P.A. Istiqlal    Ciliwung  -6,168,254  106,831,350   
2020-01-01 00:40:00  P.A. Istiqlal    Ciliwung  -6,168,254  106,831,350   
2020-01-01 01:10:00  P.A. Istiqlal    Ciliwung  -6,168,254  106,831,350   
...                              ...       ...         ...          ...   
2020-11-02 00:00:00                0         0           0            0   
2020-11-03 00:00:00                0         0           0            0   
2020-12-01 00:00:00                0         0           0            0   
2020-12-02 00:00:00                0         0           0            0   
2020-12-03 00:00:00                0         0           0            0   

                     tinggi_air status_siaga  curah_hujan  
2019-12-31 23:40:00      1850.0       N

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers

# Menambahkan kolom status_siaga sebagai kategori
label_encoder = LabelEncoder()
df_combined['status_siaga_encoded'] = label_encoder.fit_transform(df_combined['status_siaga'].astype(str))

# Menyiapkan data untuk model
X = df_combined[['tinggi_air', 'curah_hujan']].values
y = df_combined['status_siaga_encoded'].values

# Normalisasi data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data untuk LSTM (jumlah sample, jumlah timestep, jumlah fitur)
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Data splitting
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Model LSTM
num_classes = len(np.unique(y))
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model Training
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

# Evaluasi model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

# Prediksi untuk suatu timestamp tertentu
new_data = np.array([[2000, 31.5]])
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = new_data_scaled.reshape((1, 1, new_data_scaled.shape[1]))  # Reshape sesuai dengan format input model
predictions = model.predict(new_data_reshaped)
predicted_class = np.argmax(predictions[0])
predicted_status = label_encoder.inverse_transform([predicted_class])[0]
print(f'Predicted Status: {predicted_status}')

Epoch 1/50
55/55 [==============================] - 2s 6ms/step - loss: 1.5410 - accuracy: 0.5087 - val_loss: 1.4556 - val_accuracy: 0.4931
Epoch 2/50
55/55 [==============================] - 0s 2ms/step - loss: 1.3246 - accuracy: 0.5329 - val_loss: 1.2496 - val_accuracy: 0.4931
Epoch 3/50
55/55 [==============================] - 0s 2ms/step - loss: 1.2041 - accuracy: 0.5329 - val_loss: 1.1933 - val_accuracy: 0.4931
Epoch 4/50
55/55 [==============================] - 0s 3ms/step - loss: 1.1544 - accuracy: 0.5329 - val_loss: 1.1486 - val_accuracy: 0.4931
Epoch 5/50
55/55 [==============================] - 0s 2ms/step - loss: 1.1141 - accuracy: 0.5329 - val_loss: 1.0876 - val_accuracy: 0.4931
Epoch 6/50
55/55 [==============================] - 0s 2ms/step - loss: 1.0455 - accuracy: 0.5340 - val_loss: 1.0060 - val_accuracy: 0.4931
Epoch 7/50
55/55 [==============================] - 0s 2ms/step - loss: 0.9804 - accuracy: 0.5790 - val_loss: 0.9164 - val_accuracy: 0.6774
Epoch 8/50
55/55 [==